# Convolutional Neural Network

### Importing the libraries

In [17]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [18]:
tf.__version__

'2.10.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

### Preprocessing the Test set

In [19]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip = True,
                                   validation_split=0.2)  # val 20%

test_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_path = "Train_images"
train_set = train_datagen.flow_from_directory(train_path, 
                                               target_size=(224, 224), 
                                               color_mode='rgb',
                                               batch_size=32, 
                                               class_mode='categorical',
                                               shuffle=True,
                                               subset = 'training') 

test_set = test_datagen.flow_from_directory(train_path, 
                                           target_size=(224, 224), 
                                           color_mode='rgb',
                                           batch_size=32, 
                                           class_mode='categorical',
                                           shuffle=False,
                                           subset = 'validation')

Found 23037 images belonging to 5 classes.
Found 5756 images belonging to 5 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [20]:
import tensorflow.keras.layers as layers

In [127]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [128]:
cnn = tf.keras.models.Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(512, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')
])

In [129]:
cnn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 222, 222, 64)      1792      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 111, 111, 64)     0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 109, 109, 128)     73856     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 54, 54, 128)      0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 52, 52, 256)       295168    
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 26, 26, 256)     

## Part 3 - Training the CNN

### Compiling the CNN

In [130]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [182]:
from tensorflow.keras.callbacks import TensorBoard
log_dir = "logs"  # Replace with your preferred directory
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


In [183]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Define your model

# Define callbacks
checkpoint = ModelCheckpoint("best_model.h5", monitor="val_loss", save_best_only=True)
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=5, min_lr=1e-6)

In [197]:
history = cnn.fit(
    train_set,
    epochs=250,
    validation_data=test_set,
    callbacks=[checkpoint, early_stopping, reduce_lr,tensorboard_callback]
)

Epoch 1/250
720/720 [==============================] - 174s 241ms/step - loss: 0.4256 - accuracy: 0.8394 - val_loss: 0.3361 - val_accuracy: 0.8706 - lr: 0.0010
Epoch 2/250
720/720 [==============================] - 170s 236ms/step - loss: 0.4160 - accuracy: 0.8452 - val_loss: 0.3276 - val_accuracy: 0.8737 - lr: 0.0010
Epoch 3/250
720/720 [==============================] - 171s 237ms/step - loss: 0.4168 - accuracy: 0.8436 - val_loss: 0.3359 - val_accuracy: 0.8805 - lr: 0.0010
Epoch 4/250
720/720 [==============================] - 172s 238ms/step - loss: 0.4138 - accuracy: 0.8466 - val_loss: 0.3514 - val_accuracy: 0.8678 - lr: 0.0010
Epoch 5/250
720/720 [==============================] - 171s 237ms/step - loss: 0.4009 - accuracy: 0.8541 - val_loss: 0.3208 - val_accuracy: 0.8862 - lr: 0.0010
Epoch 6/250
720/720 [==============================] - 174s 241ms/step - loss: 0.4039 - accuracy: 0.8501 - val_loss: 0.2983 - val_accuracy: 0.8876 - lr: 0.0010
Epoch 7/250
720/720 [===================

In [196]:
%load_ext tensorboard
%tensorboard --logdir logs


Reusing TensorBoard on port 6006 (pid 33624), started 0:23:06 ago. (Use '!kill 33624' to kill it.)

In [199]:
evaluation = cnn.evaluate(test_set)

# Print the loss and accuracy
print("Test loss:", evaluation[0])
print("Test accuracy:", evaluation[1])

180/180 [==============================] - 10s 54ms/step - loss: 0.2213 - accuracy: 0.9142
Test loss: 0.22126680612564087
Test accuracy: 0.9141765236854553


In [200]:
best_val_accuracy = max(history.history['val_accuracy'])
print("Best Validation Accuracy:", best_val_accuracy)

Best Validation Accuracy: 0.9145239591598511


## Part 4 - Making a single prediction

In [201]:
from tensorflow.keras.models import load_model

best_model = load_model("best_model.h5")


In [215]:
import numpy as np
from tensorflow.keras.preprocessing import image


class_labels = ['Cigar-shaped smooth',   'In between smooth','completely round smooth','edge-on', 'spiral']

path = "Cigar-shaped smooth"
image_code = "147305"

img_path = f"Train_images/{path}/{image_code}.jpg"


img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Normalize pixel values to [0, 1]

predictions = best_model.predict(img_array)
predicted_class_index = np.argmax(predictions)
predicted_class_label = class_labels[predicted_class_index]

print("Predicted class:", predicted_class_label)
print("Prediction probabilities:", predictions)


1/1 [==============================] - 0s 16ms/step
Predicted class: Cigar-shaped smooth
Prediction probabilities: [[5.5301964e-01 4.1012773e-01 7.4697056e-07 3.6497749e-02 3.5415587e-04]]


In [193]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load your trained model
model = cnn  # Load your model here

# Load an image for prediction
path = "edge-on"
image_code = "101232"

img_path = f"Train_images/{path}/{image_code}.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

# Preprocess the image
img_array = img_array / 255.0  # Normalize pixel values to [0, 1]

# Make a prediction
predictions = model.predict(img_array)

# Convert predictions to class labels

predicted_class_index = np.argmax(predictions)
predicted_class_label = class_labels[predicted_class_index]

print("Predicted class:", predicted_class_label)
print("Prediction probabilities:", predictions)


1/1 [==============================] - 0s 13ms/step
Predicted class: In between smooth
Prediction probabilities: [[1.3218604e-01 6.7485460e-05 1.7512243e-10 8.5531127e-01 1.2435321e-02]]


In [152]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('Train_images/spiral/100951.jpg', target_size = (224, 224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
train_set.class_indices
if result[0][0] == 1:
  prediction = 'Cigar-shaped smooth'
elif result[0][1] ==1:
  prediction = 'completely round smooth'
elif result[0][2] ==1:
  prediction = 'edge-on'
elif result[0][3] ==1:
  prediction = 'In between smooth'
else:
  prediction = 'spiral'

print(result[0])

1/1 [==============================] - 0s 12ms/step
[0. 0. 0. 0. 1.]


In [146]:
print(prediction)

spiral
